In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Set up the notebook environment
!pip install catboost xgboost lightgbm scikit-learn
import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.io
from scipy.io import savemat
from scipy import stats
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.signal import butter, filtfilt, welch, decimate
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from scipy.io import loadmat

In [ ]:
data_train = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/Ted Yang/Copy of raw_training_data.mat')
data_test = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/Ted Yang/Copy of leaderboard_data.mat')

In [ ]:
"""glove1 = np.vstack(data_glove[0])
glove2 = np.vstack(data_glove[1])
glove3 = np.vstack(data_glove[2])

ecog1 = np.vstack(data_ecog[0])
ecog2 = np.vstack(data_ecog[1])
ecog3 = np.vstack(data_ecog[2])

test_ecog1 = np.vstack(test_ecog[0])
test_ecog2 = np.vstack(test_ecog[1])
test_ecog3 = np.vstack(test_ecog[2])"""

'glove1 = np.vstack(data_glove[0])\nglove2 = np.vstack(data_glove[1])\nglove3 = np.vstack(data_glove[2])\n\necog1 = np.vstack(data_ecog[0])\necog2 = np.vstack(data_ecog[1])\necog3 = np.vstack(data_ecog[2])\n\ntest_ecog1 = np.vstack(test_ecog[0])\ntest_ecog2 = np.vstack(test_ecog[1])\ntest_ecog3 = np.vstack(test_ecog[2])'

In [ ]:
data_glove = data_train['train_dg']
data_ecog = data_train['train_ecog']
test_ecog = data_test['leaderboard_ecog']

train_dg = []
train_ecog = []

# Function to load the entire dataset as training data
def load_data(data):
    return data[0]  # Assuming each 'data' item is wrapped in an outer array

# Loop over each subject's data for glove and ECoG data
for glove_data, ecog_data in zip(data_glove, data_ecog):
    train_dg.append(load_data(glove_data))
    train_ecog.append(load_data(ecog_data))

# Now train_dg and train_ecog contain all the data for each subject as training data

# Calculate total number of samples in the training set for the first subject (as an example)
total_samples_train_ecog = len(train_ecog[0])

print(f'The total number of samples in the training set is {total_samples_train_ecog}.')


The total number of samples in the training set is 300000.


In [ ]:
ecog_test = []

for ecog_t in test_ecog:
    ecog_test.append(load_data(ecog_t))

total_samples_test_ecog = len(ecog_test[0])

print(total_samples_test_ecog)

147500


In [ ]:
def filter_data(raw_eeg, fs=1000):
    """
    Input:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)

    Output:
      clean_data (samples x channels): the filtered signal
    """

    #lowcut and highcut for the bandpass filter
    lowcut = 0.15
    highcut = 200.0

    #Butterworth bandpass filter
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(4, [low, high], btype='band', analog=False)

    #apply filter to raw ECoG data
    clean_data = filtfilt(b, a, raw_eeg, axis=0)

    return clean_data


In [ ]:
#window and step sizes
window_length = 100
step_size = 50

#number of feature windows M for the full dataset, training, and testing sets
#M_full = 1 + (total_samples_full_ecog - window_length) // step_size
M_train = 1 + (total_samples_train_ecog - window_length) // step_size
#M_test = 1 + (total_samples_test_ecog - window_length) // step_size

#print(f'There are {M_full} windows using all the data.')
print(f'There are {M_train} windows training data.')
#print(f'There are {M_test} windows testing data.')

There are 5999 windows training data.


In [ ]:
def get_features(filtered_window, fs=1000):
    """
    Inputs:
        filtered_window (window_samples x channels): the window of the filtered ECoG signal
        fs: sampling rate

    Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    # Make sure nperseg does not exceed window size
    nperseg = min(filtered_window.shape[0], 256)

    # Basic statistical features
    mean = np.mean(filtered_window, axis=0)
    variance = np.var(filtered_window, axis=0)
    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential

    # Compute the Power Spectral Density (PSD) for each channel
    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)

    # Define frequency bands
    bands = {
        'alpha': (8, 12),
        #'beta': (13, 30),
        #'theta': (4, 7),  # Adding an extra band as an example
        #'low_gamma': (30, 50),  # Another example band
        'custom_band1': (18, 24),
        'custom_band2': (75, 115),
        'custom_band3': (125, 159),
        'custom_band4': (159, 175)
    }

    # Initialize a dictionary to hold the power for each band
    band_powers = {band: np.zeros(psd.shape[0]) for band in bands}

    # Calculate power for each band
    for band, (low, high) in bands.items():
        freq_mask = (freqs >= low) & (freqs <= high)
        band_powers[band] = np.sum(psd[:, freq_mask], axis=1)

    # Collect all features
    all_features = [mean, variance, lmp] + list(band_powers.values())
    features = np.vstack(all_features).T

    return features


In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length_ms, window_overlap_ms):
    """
    Inputs:
        raw_ecog (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length_ms: the window's length in milliseconds
        window_overlap_ms: the window's overlap in milliseconds

    Output:
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
    """
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    #filter_data function placeholder
    filtered_ecog = raw_ecog

    total_samples = raw_ecog.shape[0]
    num_windows = 1 + (total_samples - window_length) // step_size

    feature_list = []

    for start_idx in tqdm(range(0, total_samples - window_length + 1, step_size)):
        end_idx = start_idx + window_length
        window = filtered_ecog[start_idx:end_idx, :]

        #features for this window
        window_features = get_features(window, fs)

        #append features to list
        feature_list.append(window_features.flatten())

    #stack to get 2D array (num_windows x features)
    all_feats = np.array(feature_list)

    return all_feats

In [ ]:
def create_R_matrix(features, N=3):
    """
    Input:
        features: The 2D array of features (num_windows x num_features_per_window).
        N: The number of previous windows to include for each prediction instance.

    Output:
        R: The response matrix.
    """
    #first N-1 rows to beginning of features matrix
    adjusted_features = np.vstack([features[:N-1], features])

    #number of total instances after adjustment
    M_prime = adjusted_features.shape[0] - (N - 1)

    #initialize response matrix R
    num_features = features.shape[1]
    R = np.zeros((M_prime, N * num_features + 1))  # +1 for the intercept term

    #fill in R
    for i in range(M_prime):
        #extract N consecutive windows of features
        consecutive_features = adjusted_features[i:i+N].flatten()
        #fill corresponding row in R, adding 1 as the last column
        R[i, 1:] = consecutive_features
        R[i, 0] = 1  # Intercept term

    return R

In [ ]:
train_ecog_1 = train_ecog[0]
train_ecog_2 = train_ecog[1]
train_ecog_3 = train_ecog[2]

train_glove_1 = train_dg[0]
train_glove_2 = train_dg[1]
train_glove_3 = train_dg[2]

In [ ]:
#feature extraction for each subject
feature_1 = get_windowed_feats(train_ecog_1, 1000, 100, 50)
feature_2 = get_windowed_feats(train_ecog_2, 1000, 100, 50)
feature_3 = get_windowed_feats(train_ecog_3, 1000, 100, 50)

num_target_windows = feature_1.shape[0]
num_target_windows

100%|██████████| 5999/5999 [00:10<00:00, 597.48it/s]


5999

In [ ]:
#get R matrix
R_1 = create_R_matrix(feature_1, N=3)
R_2 = create_R_matrix(feature_2, N=3)
R_3 = create_R_matrix(feature_3, N=3)

In [ ]:
def get_windowed_target(glove_data, window_length_ms, window_overlap_ms, fs=1000):
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    num_windows = 1 + (glove_data.shape[0] - window_length) // step_size
    targets = []

    for start_idx in range(0, glove_data.shape[0] - window_length + 1, step_size):
        end_idx = start_idx + window_length
        window = glove_data[start_idx:end_idx, :]
        targets.append(np.mean(window, axis=0))  # Example: mean over the window

    return np.array(targets)

# Windowing target glove data
target_1 = get_windowed_target(train_glove_1, 100, 50)
target_2 = get_windowed_target(train_glove_2, 100, 50)
target_3 = get_windowed_target(train_glove_3, 100, 50)


In [ ]:
# XGBoost Model
xgb_model1 = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb_model1.fit(R_1[:, 1:], target_1)

xgb_model2 = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb_model2.fit(R_2[:, 1:], target_2)

xgb_model3 = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb_model3.fit(R_3[:, 1:], target_3)

# CatBoost Model
cat_model1 = CatBoostRegressor(iterations=100, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model1.fit(R_1[:, 1:], target_1)

cat_model2 = CatBoostRegressor(iterations=100, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model2.fit(R_2[:, 1:], target_2)

cat_model3 = CatBoostRegressor(iterations=100, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model3.fit(R_3[:, 1:], target_3)

In [ ]:
def linear_filter(R_matrix, train_data, lambda_reg=0.01):
    num_samples_features = R_matrix.shape[0]
    num_samples_flexion = train_data.shape[0]

    # Downsample train_data if it doesn't match the number of feature windows in R_matrix
    if num_samples_features != num_samples_flexion:
        downsample_factor = num_samples_flexion // num_samples_features
        downsample_factor = max(downsample_factor, 1)  # Ensure downsample_factor at least 1
        downsize_data = decimate(train_data, downsample_factor, axis=0)

        # Ensure downsize_data has same number of samples as R_matrix
        if downsize_data.shape[0] > num_samples_features:
            downsize_data = downsize_data[:num_samples_features]
    else:
        downsize_data = train_data

    # Regularized linear filter f using matrix multiplication
    # p1 = (R^T R + lambda * I)^(-1)
    lambda_identity = lambda_reg * np.eye(R_matrix.shape[1])  # Lambda times identity matrix
    p1 = np.linalg.inv(np.matmul(R_matrix.T, R_matrix) + lambda_identity)

    # p2 = R^T Y
    p2 = np.matmul(R_matrix.T, downsize_data)

    # f = p1 * p2
    lf = np.matmul(p1, p2)

    return downsize_data, lf

In [ ]:
test_ecog_1 = ecog_test[0]
test_ecog_2 = ecog_test[1]
test_ecog_3 = ecog_test[2]

In [ ]:
#feature extraction of test set
test_feature_1 = get_windowed_feats(test_ecog_1, 1000, 100, 50)
test_feature_2 = get_windowed_feats(test_ecog_2, 1000, 100, 50)
test_feature_3 = get_windowed_feats(test_ecog_3, 1000, 100, 50)

100%|██████████| 2949/2949 [00:02<00:00, 1273.86it/s]


In [ ]:
#R matrix for test set
test_R_1 = create_R_matrix(test_feature_1, N=3)
test_R_2 = create_R_matrix(test_feature_2, N=3)
test_R_3 = create_R_matrix(test_feature_3, N=3)

In [ ]:
xgb_preds_1 = xgb_model1.predict(test_R_1[:, 1:])
xgb_preds_2 = xgb_model2.predict(test_R_2[:, 1:])
xgb_preds_3 = xgb_model3.predict(test_R_3[:, 1:])

cat_preds_1 = cat_model1.predict(test_R_1[:, 1:])
cat_preds_2 = cat_model2.predict(test_R_2[:, 1:])
cat_preds_3 = cat_model3.predict(test_R_3[:, 1:])

In [ ]:
xgb_preds_1, xgb_preds_2, xgb_preds_3

(array([[-0.276246  ,  0.08683507, -0.34016916, -0.64499956, -0.23695357],
        [-0.34465128, -0.05013668, -0.34140992, -0.6199694 , -0.21154611],
        [-0.37306714, -0.04871317, -0.3389657 , -0.5176112 , -0.10618192],
        ...,
        [-0.34801325, -0.12616602, -0.18780437,  1.0660299 ,  0.17785095],
        [-0.25814933,  0.35950476, -0.06475341,  1.6907852 ,  0.9869624 ],
        [-0.28077778,  0.10425006, -0.22813179,  0.51824844,  0.06467355]],
       dtype=float32),
 array([[-0.04120429, -0.36518627,  0.20164004, -0.3572953 , -0.22289106],
        [ 0.25539473, -0.4180713 , -0.27459094, -0.42072293, -0.11551061],
        [ 0.2071678 , -0.35038176,  0.2519739 , -0.37220296, -0.16499338],
        ...,
        [-0.41464287,  0.7588389 ,  0.32999957,  1.3957771 ,  0.1852043 ],
        [-0.5811099 ,  0.07621998,  0.20146127,  1.1077783 ,  0.11789968],
        [-0.6689643 , -0.26714423,  0.59482944,  1.4283397 , -0.01416012]],
       dtype=float32),
 array([[-0.38762963,  0.1

In [ ]:
cat_preds_1, cat_preds_2, cat_preds_3

(array([[-0.06041115,  0.35977187, -0.19012601, -0.49239783, -0.30334726],
        [-0.05299863, -0.1404011 , -0.35123398, -0.43950861, -0.25827014],
        [-0.1406614 ,  0.10818122, -0.20223093, -0.33023922, -0.24376433],
        ...,
        [-0.28485534, -0.13003698, -0.10486655,  0.57499037,  0.36806719],
        [-0.51508431,  0.23749318,  0.10903646,  1.26780971,  0.59504584],
        [-0.32175139,  0.15967663,  0.02694744,  0.71875018,  0.64371652]]),
 array([[-0.14544039, -0.15546299, -0.08833927, -0.25794168, -0.06521212],
        [-0.0406662 , -0.10147094, -0.06423673, -0.29735683, -0.07199219],
        [-0.08121671, -0.16344162, -0.06932305, -0.25132591, -0.07708059],
        ...,
        [-0.28603777,  0.31477863,  0.5670668 ,  0.58893636,  0.21407428],
        [-0.49994379, -0.00563304,  0.45446722,  0.96317203,  0.31255107],
        [-0.65787673,  0.10330456,  0.63666155,  0.77861309,  0.21565628]]),
 array([[-0.24644965,  0.02826703,  0.01518259,  0.47896534,  0.116646

In [ ]:
from scipy.io import savemat

In [ ]:
original_length = xgb_preds_1.shape[0]
desired_length = test_ecog_1.shape[0]

In [ ]:
def interpolate(prediction, original_length, desired_length):
    # Step 1: Create a time array for the original prediction
    original_time = np.linspace(0, 1, original_length)

    # Step 2: Create a time array for the desired length
    desired_time = np.linspace(0, 1, desired_length)

    # Step 3: Interpolate each column separately using cubic spline
    interpolated_prediction = np.zeros((desired_length, prediction.shape[1]))
    for i in range(prediction.shape[1]):
        cs = CubicSpline(original_time, prediction[:, i])
        interpolated_prediction[:, i] = cs(desired_time)

    return interpolated_prediction

In [ ]:
inter_xgb_prediction1 = interpolate(xgb_preds_1, original_length, desired_length)
inter_xgb_prediction2 = interpolate(xgb_preds_2, original_length, desired_length)
inter_xgb_prediction3 = interpolate(xgb_preds_3, original_length, desired_length)

## **XGBoost - 0.4368**

**Training Correaltion:**

([0.42975486277270897,
  0.576615967656113,
  0.14138890739763021,
  0.44750700729701787,
  0.10267421165647812],
 [0.5077436689148577,
  0.18126293860768045,
  0.21879286202665796,
  0.5181947857825426,
  0.07912924578167839],
 [0.5731551697779543,
  0.3824455186605641,
  0.46025726563488767,
  0.5024670634360416,
  0.37533030506974796])

In [ ]:
#XGBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_xgb_prediction1
predicted_dg[1,0] = inter_xgb_prediction2
predicted_dg[2,0] = inter_xgb_prediction3

savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
print(predicted_dg)

[[array([[-0.27624601,  0.08683507, -0.34016916, -0.64499956, -0.23695357],
         [-0.26983416,  0.07929754, -0.33916461, -0.64863877, -0.24066889],
         [-0.26390119,  0.07202392, -0.33822311, -0.65204225, -0.24414392],
         ...,
         [-0.28036208,  0.15985895, -0.20743528,  0.62063515,  0.1863392 ],
         [-0.2805888 ,  0.13268947, -0.21759095,  0.57012915,  0.12665753],
         [-0.28077778,  0.10425006, -0.22813179,  0.51824844,  0.06467355]])]
 [array([[-0.04120429, -0.36518627,  0.20164004, -0.3572953 , -0.22289106],
         [-0.0284904 , -0.37109572,  0.1625483 , -0.36141238, -0.21645006],
         [-0.01612072, -0.37673463,  0.12505345, -0.36537857, -0.21023879],
         ...,
         [-0.64909381, -0.29060688,  0.55373356,  1.36807235, -0.01869973],
         [-0.65876579, -0.27942233,  0.57393747,  1.39752587, -0.0165897 ],
         [-0.66896433, -0.26714423,  0.59482944,  1.42833972, -0.01416012]])]
 [array([[-0.38762963,  0.1671651 , -0.16644694,  0.6387

## **CatBoost - 0.4426**

**Training Correaltion:**

([0.42110667057890294,
  0.5955357296984822,
  0.16680256550611047,
  0.49621501159105874,
  0.09428410688813813],
 [0.47397291001606884,
  0.18774692153335576,
  0.28783016125629185,
  0.4229055985315709,
  0.3153480826008141],
 [0.5941576953268893,
  0.3906184545554212,
  0.4958015312549799,
  0.5647895824977968,
  0.42089092458207944])

In [ ]:
inter_cat_prediction1 = interpolate(cat_preds_1, original_length, desired_length)
inter_cat_prediction2 = interpolate(cat_preds_2, original_length, desired_length)
inter_cat_prediction3 = interpolate(cat_preds_3, original_length, desired_length)

In [ ]:
#CatBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_cat_prediction1
predicted_dg[1,0] = inter_cat_prediction2
predicted_dg[2,0] = inter_cat_prediction3

savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
predicted_dg

array([[array([[-0.06041115,  0.35977187, -0.19012601, -0.49239783, -0.30334726],
               [-0.05781198,  0.32677521, -0.20147538, -0.49117021, -0.30101811],
               [-0.05534284,  0.29502652, -0.21239246, -0.48996383, -0.29876953],
               ...,
               [-0.35158756,  0.19089701,  0.05059249,  0.79897031,  0.65515861],
               [-0.33697237,  0.17568126,  0.03905837,  0.7596461 ,  0.64962879],
               [-0.32175139,  0.15967663,  0.02694744,  0.71875018,  0.64371652]])],
       [array([[-0.14544039, -0.15546299, -0.08833927, -0.25794168, -0.06521212],
               [-0.13980681, -0.15112746, -0.08707746, -0.2604296 , -0.065518  ],
               [-0.13435946, -0.14696669, -0.08585723, -0.2628311 , -0.06581388],
               ...,
               [-0.65217878,  0.07686935,  0.62465665,  0.79829566,  0.21370711],
               [-0.65503177,  0.08978506,  0.63061481,  0.78859071,  0.21457128],
               [-0.65787673,  0.10330456,  0.63666155, 

In [ ]:
pred = loadmat('predictions.mat')
print(pred['predicted_dg'])


[[array([[-0.06041115,  0.35977187, -0.19012601, -0.49239783, -0.30334726],
         [-0.05781198,  0.32677521, -0.20147538, -0.49117021, -0.30101811],
         [-0.05534284,  0.29502652, -0.21239246, -0.48996383, -0.29876953],
         ...,
         [-0.35158756,  0.19089701,  0.05059249,  0.79897031,  0.65515861],
         [-0.33697237,  0.17568126,  0.03905837,  0.7596461 ,  0.64962879],
         [-0.32175139,  0.15967663,  0.02694744,  0.71875018,  0.64371652]])]
 [array([[-0.14544039, -0.15546299, -0.08833927, -0.25794168, -0.06521212],
         [-0.13980681, -0.15112746, -0.08707746, -0.2604296 , -0.065518  ],
         [-0.13435946, -0.14696669, -0.08585723, -0.2628311 , -0.06581388],
         ...,
         [-0.65217878,  0.07686935,  0.62465665,  0.79829566,  0.21370711],
         [-0.65503177,  0.08978506,  0.63061481,  0.78859071,  0.21457128],
         [-0.65787673,  0.10330456,  0.63666155,  0.77861309,  0.21565628]])]
 [array([[-0.24644965,  0.02826703,  0.01518259,  0.4789